In [1]:
import pandas as pd
import os 

In [2]:
root = 'merging_labels/'
fnames = os.listdir(root)
fulls = [os.path.join(root,i) for i in fnames]
fields = ['image', 'choice']
merge_data = pd.concat([pd.read_csv(f, skipinitialspace=True, usecols=fields) for f in fulls])

In [3]:
merge_data

,image,choice
0,GX010049_frame10350,Very safe
1,GX010049_frame10800,Very safe
2,GX010049_frame11100,Very safe
3,GX010049_frame11700,Very safe
4,GX010049_frame11850,Very safe
...,...,...
994,/data/local-files/?d=Users/admin/environments/...,NaN
995,/data/local-files/?d=Users/admin/environments/...,Safe
996,/data/local-files/?d=Users/admin/environments/...,Safe
997,/data/local-files/?d=Users/admin/environments/...,Very safe


In [4]:
df = merge_data
df['image'] = df['image'].apply(lambda x: x.split('/')[-1] if x[0]=='/' else x+'.jpg')

In [5]:
# df = df.drop(columns=['annotation_id', 'id', 'updated_at'])

In [5]:
import numpy as np
def certainty_score(score_list):
    arr = np.array(score_list)
    unique_values, counts = np.unique(arr, return_counts=True)
    index_max = np.argmax(counts)
    return unique_values[index_max], counts[index_max]/len(score_list)

In [6]:
unique_image_list = pd.unique(df['image']).tolist()

In [47]:
merge_score_list = []
gt_label_list = []
training_img_list = []
for img in unique_image_list:
    rows = df.loc[df['image'] == img]
    score_list = rows['choice'].tolist()
    if len(score_list)==3:
        label = certainty_score(score_list)[0]
        final_score = float(certainty_score(score_list)[-1])
        
        merge_score_list.append(final_score)
        gt_label_list.append(label)
        training_img_list.append(img)

In [48]:
len(training_img_list),len(merge_score_list), len(gt_label_list)

(899, 899, 899)

In [49]:
totally_agree = 0
mostly_agree = 0
disagree = 0
for s in merge_score_list:
    if s==1:
        totally_agree+=1
    elif s>0.5 and s<0.7:
        mostly_agree+=1
    else:
        disagree+=1
print(totally_agree/len(merge_score_list), mostly_agree/len(merge_score_list), disagree/len(merge_score_list))

0.23581757508342602 0.5962180200222469 0.16796440489432704


In [50]:
data = {'image': training_img_list,
        'label': gt_label_list}
gt_df = pd.DataFrame(data)
gt_df.to_csv('gt_df.csv', index=False)

In [52]:
gt_df.head(-5)

,image,label
0,GX010049_frame10350.jpg,Safe
1,GX010049_frame10800.jpg,Safe
2,GX010049_frame11100.jpg,Safe
3,GX010049_frame11700.jpg,Safe
4,GX010049_frame11850.jpg,Safe
...,...,...
889,GX080055_frame2550.jpg,Unsafe
890,GX080055_frame300.jpg,Safe
891,GX080055_frame3300.jpg,Unsafe
892,GX080055_frame4050.jpg,Unsafe


In [53]:
# simplify gt_df
for i in range(len(gt_df)):
    l = gt_df['label'].loc[i]
    if l == 'Very safe':
        gt_df['label'].loc[i]='Safe'
    elif l == 'Very unsafe':
        gt_df['label'].loc[i]='Unsafe'
    else:
        continue

In [55]:
gt_df.to_csv('gt_df_simple.csv', index=False)